In [1]:
'''
Evaluate the performance of search algorithms 
Collect scores across all prompts and trials, and compute the overall statistics.
'''

import os, psutil, gc
import time 
import json
import pprint

from collections import defaultdict
import random

import numpy as np
np.set_printoptions(precision=4)
from scipy.stats import ttest_rel

In [2]:
from sal.config import Config

from datasets import load_dataset

# from core import best_of_n
from utils.load_data import load_data_prm800k
from utils import grader 
from utils import grader2
from utils import parser

In [3]:
# base_dir
base_dir = '/groups/kjun/tnn/datasets/'

# dataset path
data_dir = base_dir + "/prm800k/math_splits"

# llm and prm path
llm_dir = base_dir + "/Llama-3.2-1B-Instruct-GGUF/Llama-3.2-1B-Instruct.Q4_K_M.gguf"
prm_dir = base_dir + "/Llama3.1-8B-PRM-Deepseek-Data-GGUF/Llama3.1-8B-PRM-Deepseek-Data.Q4_K_M.gguf"

llm_tokenizer_dir = base_dir + "/Llama-3.2-1B-Instruct"
prm_tokenizer_dir = base_dir + "/Llama3.1-8B-PRM-Deepseek-Data"

In [4]:
#  load data 
data_by_levels = load_data_prm800k(data_dir)


# ds_completions = load_completions(completions_dir)

# load random_seeds     
# random_seeds = np.loadtxt("random_seeds.txt").astype("int64")
# random_seeds = [int(seed) for seed in random_seeds]

1: 43
2: 90
3: 105
4: 128
5: 134


In [5]:
import signal

In [54]:
class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException()

def run_with_timeout(fn_extract_answer, fn_grade, completion, gt_answer, timeout=2):
    # Set the signal handler for SIGALRM
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout)  # Schedule an alarm after `timeout` seconds
    try:
        c_answer = fn_extract_answer(completion, 'math')
        result = fn_grade(c_answer, gt_answer)
    except TimeoutException:
        print(f"Timeout: {completion}")
        c_answer = None
        result = None
    finally:
        signal.alarm(0)  # Cancel alarm if function returns early
    return c_answer, result

def evaluate_correctness_hf(data_dir, level, n=8, limit_budget=False):

    # dataset = load_dataset(dataset_name, name=config_name, split=dataset_split, cache_dir=data_dir)
    dataset = load_dataset("json", data_files = data_dir, split='train')
    dataset_by_level = dataset.filter(lambda example: example['level'] == int(level))

    passn_ncomps = np.zeros((len(dataset_by_level)))
    passn_lengths = np.zeros((len(dataset_by_level)))

    pass1b_ncomps = np.zeros((len(dataset_by_level)))
    pass1b_lengths = np.zeros((len(dataset_by_level)))
    for q_idx, data in enumerate(dataset_by_level):
        passn_completions = data["completions"][:n]
        pass1b_completions = data["completions"]

        passn_total_len = 0 
        for cidx, completion in enumerate(passn_completions):
            passn_total_len += len(completion.split("\n\n"))
            
        pass1b_total_len = 0
        for cidx, completion in enumerate(pass1b_completions):
            pass1b_total_len += len(completion.split("\n\n"))
        

        passn_ncomps[q_idx] = len(passn_completions)
        pass1b_ncomps[q_idx] = len(pass1b_completions)
        
        passn_lengths[q_idx] = passn_total_len/len(passn_completions) if len(passn_completions) > 0 else 0
        pass1b_lengths[q_idx] = pass1b_total_len/len(pass1b_completions) if len(pass1b_completions) > 0 else 0

        
    return passn_ncomps, pass1b_ncomps, passn_lengths, pass1b_lengths


# general params
config = Config()
config.agg_strategy = 'last'
config.n = 8
config.beam_width = 2
config.lookahead = 0
config.num_iterations = 10
config.sort_completed = False

# diverse_select params
config.lam = 10
config.normalize_embeds = True

level = '4'
# num_questions = len(data_by_levels[level])
# num_questions = 1
num_trials = 2
# print(f"num_questions = {num_questions}")

config_name = "bob--n-8--d-40--level-4--v11"
config_name = "bob--v11--n-40--d-40--level-4"

all_passn_ncomps = []
all_pass1b_ncomps = []
all_passn_lengths = []
all_pass1b_lengths = []
for trial_idx in range(num_trials):
    passn_ncomps, pass1b_ncomps, passn_lengths, pass1b_lengths = \
        evaluate_correctness_hf(f"results/{config_name}/{config_name}--trial-{trial_idx}.jsonl", level, config.n, limit_budget=False)
    # print(passn_ncomps)
    
    all_passn_ncomps.append(passn_ncomps)
    all_pass1b_ncomps.append(pass1b_ncomps)
    all_passn_lengths.append(passn_lengths)
    all_pass1b_lengths.append(pass1b_lengths)

# print(passn_ncomps)
# print(len(passn_ncomps))
all_passn_ncomps = np.concatenate(all_passn_ncomps)
all_pass1b_ncomps = np.concatenate(all_pass1b_ncomps)

all_passn_lengths = np.concatenate(all_passn_lengths)
all_pass1b_lengths = np.concatenate(all_pass1b_lengths)

passn_ncomps_mean = np.mean(all_passn_ncomps)
pass1b_ncomps_mean = np.mean(all_pass1b_ncomps)
passn_lengths_mean = np.mean(all_passn_lengths)
pass1b_lengths_mean = np.mean(all_pass1b_lengths)


passn_ncomps_std = np.std(all_passn_ncomps, ddof=1)/np.sqrt(num_trials*128) # 128 is number of prompts for level 4 
pass1b_ncomps_std = np.std(all_pass1b_ncomps, ddof=1)/np.sqrt(num_trials*128)
passn_lengths_std = np.std(all_passn_lengths, ddof=1)/np.sqrt(num_trials*128)
pass1b_lengths_std = np.std(all_pass1b_lengths, ddof=1)/np.sqrt(num_trials*128)

# print(passn_correctness)
# print(pass1b_correctness)
# print(weighted1b_correctness)
# print(pass1b_correctness_mean)
# print(pass1b_correctness_std)

print(f"passn_ncomps: {passn_ncomps_mean:0.1f} (\u00B1{passn_ncomps_std:0.1f})")
print(f"pass1b_ncomps: {pass1b_ncomps_mean:0.1f} (\u00B1{pass1b_ncomps_std:0.1f})")
print(f"passn_lengths: {passn_lengths_mean:0.1f} (\u00B1{passn_lengths_std:0.1f})")
print(f"pass1b_lengths: {pass1b_lengths_mean:0.1f} (\u00B1{pass1b_lengths_std:0.1f})")


passn_ncomps: 8.0 (±0.0)
pass1b_ncomps: 153.1 (±4.2)
passn_lengths: 4.5 (±0.1)
pass1b_lengths: 16.6 (±0.2)
